In [ ]:
library(deSolve)
library(ggplot2)
library(scales)

# SIR model function
sir_model <- function(time, state, parameters) {
  with(as.list(c(state, parameters)), {
    dS <- -beta * S * I / N
    dI <- beta * S * I / N - gamma * I
    dR <- gamma * I
    return(list(c(dS, dI, dR)))
  })
}

In [ ]:
# Set parameters
parameters <- c(beta = 0.5, gamma = 0.1)
state <- c(S = 99000, I = 1000, R = 0)
times <- seq(0, 180, by = 1)
N <- sum(state)

In [ ]:
# Solve the differential equations
out <- ode(y = state, times = times, func = sir_model, parms = parameters)
df <- as.data.frame(out)

In [ ]:
# Plot
ggplot(df, aes(x = time)) +
  geom_line(aes(y = S, color = "Susceptible")) +
  geom_line(aes(y = I, color = "Infected")) +
  geom_line(aes(y = R, color = "Recovered")) +
  scale_color_manual(values = c("Susceptible" = "blue", "Infected" = "red", "Recovered" = "green")) +
  labs(x = "Time (Days)", y = "Number of infections", color = "") +
  theme_minimal() +
  theme(legend.position = "right")+
  scale_y_continuous(labels = comma)

In [ ]:
sirv_model <- function(time, state, parameters) {
  with(as.list(c(state, parameters)), {
    
    N <- S + I + R + V
    
    vacc_rate <- ifelse(time >= vaccine_start,
                        vaccine_coverage / vaccine_duration,
                        0)
    
    dS <- -beta * S * I / N - (vacc_rate * vacc_eff) * S
    dI <- beta * S * I / N - gamma * I
    dR <- gamma * I
    dV <- (vacc_rate * vacc_eff) * S
    
    return(list(c(dS, dI, dR, dV)))
  })
}

In [ ]:
initial_state <- c(
  S = 99000,
  I = 1000,
  R = 0,
  V = 0
)

times <- seq(0, 180, by = 1)

# differnt start days
start_days <- c(10, 30, 50, 70)


In [ ]:
# for loop
all_outputs <- data.frame()
for(i in start_days) {
  
  parameters <- c(
    beta = 0.3,
    gamma = 1/7,
    vaccine_start = i,
    vaccine_coverage = 0.5,
    vaccine_duration = 14,
    vacc_eff = 0.7
  )
  
  out <- as.data.frame(ode(y = initial_state, times = times, func = sirv_model, parms = parameters))
  
  all_outputs <- rbind(all_outputs, 
                       data.frame(time = out$time,
                                  I = out$I,
                                  vaccine_start = i))
}
all_outputs$type <- "With vaccination"

# bind with pre-vacc
pre_vacc <- df[,c(1,3)]
pre_vacc$vaccine_start <- 0 
pre_vacc$type <- "Without vaccination"
all_outputs <- rbind(pre_vacc, all_outputs)

In [ ]:
ggplot(all_outputs)+
  geom_line(aes(x = time, y = I, color = as.factor(vaccine_start), linetype = as.factor(type)))+
  theme_bw()+
  labs(color = "Vaccine start day",
      linetype = "Type")+
  scale_y_continuous(labels = comma)+
  xlab("Time (Days)")+
  ylab("Number of infections")

In [ ]:
# differnt coverage
vacc_coverage <- c(0.1, 0.3, 0.5, 0.7)

# for loop
all_outputs <- data.frame()
for(vc in vacc_coverage) {
  
  parameters <- c(
    beta = 0.3,
    gamma = 1/7,
    vaccine_start = 10,
    vaccine_coverage = vc,
    vaccine_duration = 14,
    vacc_eff = 0.7
  )
  
  out <- as.data.frame(ode(y = initial_state, times = times, func = sirv_model, parms = parameters))
  
  all_outputs <- rbind(all_outputs, 
                       data.frame(time = out$time,
                                  I = out$I,
                                  vaccine_coverage = vc))
}
all_outputs$type <- "With vaccination"
colnames(pre_vacc)[3] <- "vaccine_coverage"
all_outputs <- rbind(pre_vacc, all_outputs)

In [ ]:
ggplot(all_outputs)+
  geom_line(aes(x = time, y = I, color = as.factor(vaccine_coverage) , linetype = as.factor(type)))+
  theme_bw()+
  scale_y_continuous(labels = comma)+
  labs(color = "Vaccine coverage",
       linetype = "Type") +
  xlab("Time (Days)")+
  ylab("Number of infections")

In [ ]:
vacc_eff <- c(0.1, 0.3, 0.5, 0.7, 0.9)

# for loop
all_outputs <- data.frame()
for(ve in vacc_eff) {
  
  parameters <- c(
    beta = 0.3,
    gamma = 1/7,
    vaccine_start = 10,
    vaccine_coverage = 0.5,
    vaccine_duration = 14,
    vacc_eff         = ve
  )
  
  out <- as.data.frame(ode(y = initial_state, times = times, func = sirv_model, parms = parameters))
  
  all_outputs <- rbind(all_outputs, 
                       data.frame(time = out$time,
                                  I = out$I,
                                  vacc_eff = ve))
}
all_outputs$type <- "With vaccination"
colnames(pre_vacc)[3] <- "vacc_eff"
all_outputs <- rbind(pre_vacc, all_outputs)

In [ ]:
ggplot(all_outputs)+
  geom_line(aes(x = time, y = I, color = as.factor(vacc_eff) , linetype = as.factor(type)))+
  theme_bw()+
  scale_y_continuous(labels = comma)+
  labs(color = "Vaccine efficacy",
       linetype = "Type") +
  xlab("Time (Days)")+
  ylab("Number of infections")